# 1번 코드공유

In [ ]:
# 뭔데 나는 없는 데이터를 쓰냐?
# 일단 다 따라해보다가.. 코드공유 1도 그렇고 2도 그렇고 데이터가 따로 있는거 같아서 안됨..
# 일단 뭐 했는지만 간단하게 알아봄

In [ ]:
# xgboost 왜 난 이 사람이 한 게 왜 안되지?? 미치겠네
# 파머완이나, 따로 xgboost 실시하면 되는데 이것만 안됨
# 오류 메세지 보면 GPU와 안 맞다는데.. 하 모르겠네

# 2번 코드공유

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from pycaret.classification import *

In [ ]:
train_df = pd.read_csv('/Users/raekkkky/Desktop/DNA/Semester_2023_2/5주차/train.csv')
only_train_df = train_df.copy()
test_df = pd.read_csv('/Users/raekkkky/Desktop/DNA/Semester_2023_2/5주차/test.csv')
# original_train_df = pd.read_csv('/kaggle/input/smoker-status-prediction-using-biosignals/train_dataset.csv')
train_df = train_df.drop(['id'],axis=1)
# train_df = pd.concat([train_df, original_train_df]).sample(frac=1)
y = train_df['smoking']
train_df.info()

In [ ]:
def create_extra_features(df):
    # order the ears
    best = np.where(df['hearing(left)'] < df['hearing(right)'], 
                    df['hearing(left)'],  df['hearing(right)'])
    worst = np.where(df['hearing(left)'] < df['hearing(right)'], 
                     df['hearing(right)'],  df['hearing(left)'])
    df['hearing(left)'] = best - 1
    df['hearing(right)'] = worst - 1
    
    # order the eyes - eyesight is worst to best, and 9+ should be worst!
    df['eyesight(left)'] = np.where(df['eyesight(left)'] > 9, 0, df['eyesight(left)'])
    df['eyesight(right)'] = np.where(df['eyesight(right)'] > 9, 0, df['eyesight(right)'])
    best = np.where(df['eyesight(left)'] < df['eyesight(right)'], 
                    df['eyesight(left)'],  df['eyesight(right)'])
    worst = np.where(df['eyesight(left)'] < df['eyesight(right)'], 
                     df['eyesight(right)'],  df['eyesight(left)'])
    df['eyesight(left)'] = best
    df['eyesight(right)'] = worst
    ##
    df['Gtp'] = np.clip(df['Gtp'], 0, 300)
    df['HDL'] = np.clip(df['HDL'], 0, 110)
    df['LDL'] = np.clip(df['LDL'], 0, 200)
    df['ALT'] = np.clip(df['ALT'], 0, 150)
    df['AST'] = np.clip(df['AST'], 0, 100)
    df['serum creatinine'] = np.clip(df['serum creatinine'], 0, 3)    

create_extra_features(train_df)
create_extra_features(test_df)

In [ ]:
train_df.head()

In [ ]:
test_id = test_df['id']

In [ ]:
# 이거 함수 없길래 외부 코드에서 가져와봄

def set_frame_style(df, caption=""):
    """Helper function to set dataframe presentation style.
    """
    return df.style.background_gradient(cmap='Greens').set_caption(caption).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'Green'),
        ('font-size', '18px'),
        ('font-weight','bold')
    ]}])

In [ ]:
set_frame_style(train_df.describe())

In [ ]:
set_frame_style(train_df.head())

In [ ]:
set_frame_style(pd.DataFrame(train_df.nunique()),"Total Unique values in each column")

In [ ]:
# - No missing/null values in any of the columns
# - hearing(left), hearing(right), Urine protein, dental caries can be one hot encoded

In [ ]:
train_df.columns

In [ ]:
# Storing the categorical and numerical columns separately for different operations

In [ ]:
train_to_scale = train_df.drop(['hearing(left)', 'hearing(right)', 'Urine protein','dental caries', 'smoking'],axis =1)
train_to_ohe = train_df[['hearing(left)', 'hearing(right)', 'Urine protein','dental caries']]
test_to_scale = test_df.drop(['id','hearing(left)', 'hearing(right)', 'Urine protein','dental caries'],axis =1)
test_to_ohe = test_df[['hearing(left)', 'hearing(right)', 'Urine protein','dental caries']]

In [ ]:
# [EDA 시작] Let's see how many smoking individuals v/s non-smoking individuals

In [ ]:
train_df['smoking'].value_counts()

In [ ]:
# Synthetic data is somewhat more balanced than the original one

In [ ]:
# Distributions of Features

In [ ]:
# Observing the kde part of the Violin plot, most of them appear skewed

In [ ]:
from prettytable import PrettyTable
from itertools import combinations

correlation_matrix = train_to_scale.corr()


table = PrettyTable()
table.field_names = ["Column 1", "Column 2", "Correlation"]


for col1, col2 in combinations(correlation_matrix.columns, 2):
    correlation = correlation_matrix.at[col1, col2]
    table.add_row([col1, col2, f"{correlation:.2f}"])
table.sortby = "Correlation"

print(table)

In [ ]:
# Preprocessing [Robustscaler]

In [ ]:
from sklearn.preprocessing import RobustScaler

sc = RobustScaler()


scaled_train = pd.DataFrame(sc.fit_transform(train_to_scale),columns = train_to_scale.columns)
scaled_test = pd.DataFrame(sc.transform(test_to_scale),columns = test_to_scale.columns)

In [ ]:
set_frame_style(scaled_train.head())

In [ ]:
# One Hot Encoding the categorical columns

In [ ]:
train_to_ohe.columns

In [ ]:
ohe_train = pd.get_dummies(train_to_ohe, columns = ['hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries'])
ohe_test  = pd.get_dummies(test_to_ohe, columns = ['hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries'])

set_frame_style(ohe_train.head())

In [ ]:
for i in ohe_train.columns : 
    ohe_train[i] = ohe_train[i].apply(lambda x :0.9 if x ==1 else 0.1)

In [ ]:
ohe_train

In [ ]:
# Joining the columns after scaling and one-hot encoding

In [ ]:
ohe_train = ohe_train.reset_index(drop = True)
ohe_test = ohe_test.reset_index(drop = True)
ohe_train.head()

In [ ]:
scaled_train  = scaled_train.reset_index(drop =True)
scaled_test  = scaled_test.reset_index(drop =True)

In [ ]:
y = y.reset_index(drop = True)

In [ ]:
X_train = pd.concat([ohe_train,scaled_train,y],axis =1)
test_df = pd.concat([ohe_test,scaled_test],axis =1)
set_frame_style(X_train.head())

In [ ]:
X_train.info()

In [ ]:
test_df.info()

In [ ]:
X_train['smoking'].value_counts()

In [ ]:
# Tomek Links : Downsampling for getting balanced dataset

In [ ]:
# - Tomek links are pairs of data points where each point is from a different class,
# and the distance between them is less than or equal to a specified threshold
# - Removing data points that are part of Tomek links effectively trims the boundary of the majority class,
# making it more distinct from the minority class. This can lead to a better separation between the classes and improved model performance

In [ ]:
from imblearn.under_sampling import TomekLinks

y = X_train[['smoking']]
X = X_train.drop('smoking',axis =1)
    
def tomek_links (X,y) : 
    '''Tomek Links are pairs of instances, one from the majority class and one from the minority class,
    that are very close to each other in the feature space. They are used in the context of dealing with
    imbalanced datasets to help balance the class distribution. The key idea is to 
    identify and remove those majority class instances that are near the minority class instances'''

    tl = TomekLinks(sampling_strategy='auto')

    X_resampled, y_resampled = tl.fit_resample(X, y)
    
    return X_resampled , y_resampled

X_resampled, y_resampled = tomek_links (X,y)

In [ ]:
X_train = pd.concat([X_resampled,y_resampled], axis =1)
X_train['smoking'].value_counts()

In [ ]:
# Model Training

In [ ]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

n_splits = 10

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
auc_scores = []
best_model = None
best_auc = 0.0
X = X_resampled
y = y_resampled

params = {'learning_rate': 0.019093223225293453, 
            'colsample_bytree': 0.21329015151846925,
            'colsample_bylevel': 0.9148369225084079,
            'subsample': 0.8831564960046078,
            'reg_alpha': 1.1496763786731952e-05, 
            'reg_lambda': 7.512814356733987e-07, 
            'max_depth': 12, 
            'n_estimators': 1950,
            'min_child_weight': 21,
          'tree_method': 'gpu_hist',
          'eval_metric': 'auc',
          'booster': 'gbtree',
          'n_jobs': -1,
          'verbosity': 0}

In [ ]:
# Wrap the loop with tqdm for a progress bar
for train_index, test_index in tqdm(skf.split(X, y), total=n_splits):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = xgb.XGBClassifier(**params)

    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_pred_proba)
    auc_scores.append(auc)

    if auc > best_auc:
        best_auc = auc
        best_model = model

In [ ]:
print(best_auc)

In [ ]:
print(auc_scores)

In [ ]:
import matplotlib.pyplot as plt
feature_importances = best_model.feature_importances_
feature_names = X.columns 
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
sorted_feature_names, sorted_importance_scores = zip(*sorted_feature_importance)
plt.figure(figsize=(8, 5))
plt.barh(sorted_feature_names, sorted_importance_scores)
plt.xlabel("Feature Importance")
plt.ylabel("Feature Name")
plt.title("Feature Importance")
plt.show()

In [ ]:
y_pred = model.predict_proba(test_df)
y_pred[:,1].shape
test_id.shape
# .. 등 저장까지 진행하셨는데, 난 안됨.. ㅠ

# 3번 코드공유

In [ ]:
# Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# I copied all this from a different notebook

import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm.notebook import tqdm

import re

from functools import partial
from scipy.stats import mode

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.manifold import TSNE
import optuna
import xgboost as xgb

In [ ]:
# Read Dataset

In [ ]:
train_df = pd.read_csv('/Users/raekkkky/Desktop/DNA/Semester_2023_2/5주차/train.csv')
train_df.shape

In [ ]:
test_df = pd.read_csv('/Users/raekkkky/Desktop/DNA/Semester_2023_2/5주차/test.csv')
test_df.shape

In [ ]:
# Preprocessing

In [ ]:
def create_extra_features(df):
    # order the ears
    best = np.where(df['hearing(left)'] < df['hearing(right)'], 
                    df['hearing(left)'],  df['hearing(right)'])
    worst = np.where(df['hearing(left)'] < df['hearing(right)'], 
                     df['hearing(right)'],  df['hearing(left)'])
    df['hearing(left)'] = best - 1
    df['hearing(right)'] = worst - 1
    
    # order the eyes - eyesight is worst to best, and 9+ should be worst!
    df['eyesight(left)'] = np.where(df['eyesight(left)'] > 9, 0, df['eyesight(left)'])
    df['eyesight(right)'] = np.where(df['eyesight(right)'] > 9, 0, df['eyesight(right)'])
    best = np.where(df['eyesight(left)'] < df['eyesight(right)'], 
                    df['eyesight(left)'],  df['eyesight(right)'])
    worst = np.where(df['eyesight(left)'] < df['eyesight(right)'], 
                     df['eyesight(right)'],  df['eyesight(left)'])
    df['eyesight(left)'] = best
    df['eyesight(right)'] = worst
    ##
    df['Gtp'] = np.clip(df['Gtp'], 0, 300)
    df['HDL'] = np.clip(df['HDL'], 0, 110)
    df['LDL'] = np.clip(df['LDL'], 0, 200)
    df['ALT'] = np.clip(df['ALT'], 0, 150)
    df['AST'] = np.clip(df['AST'], 0, 100)
    df['serum creatinine'] = np.clip(df['serum creatinine'], 0, 3)    

create_extra_features(train_df)
create_extra_features(test_df)

In [ ]:
train_df.corr()['smoking'].sort_values()

In [ ]:
def bmi_(BMI):
    if BMI < 18.5: 
        return 'Underwieght'
    elif BMI >18.5 and BMI <24.99: 
        return 'Normal'
    elif BMI > 25 and BMI < 29.99: 
        return 'Overweight'
    elif BMI > 30 and BMI < 34.99: 
        return 'Obese'
    elif BMI > 35 and BMI < 39.99: 
        return 'Obese'
    else:
        return 'Obese'
train_df['bmi'] = (train_df['weight(kg)']) / ((train_df['height(cm)']/100)**2) 
train_df['bmi_cate'] = train_df['bmi'].apply(bmi_)
train_df['bmi_cate'].value_counts()
train_df = train_df.drop('bmi',axis=1)

In [ ]:
test_df['bmi'] = (test_df['weight(kg)']) / ((test_df['height(cm)']/100)**2) 
test_df['bmi_cate'] = test_df['bmi'].apply(bmi_)
test_df = test_df.drop('bmi',axis=1)

In [ ]:
train_df = pd.get_dummies(train_df, columns=['bmi_cate'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['bmi_cate'], drop_first=True)

In [ ]:
def y_gtp(gtp):
    if gtp>5 and gtp<40:
        return 1
    else:
        return 0
train_df['gtp_cate'] = train_df['Gtp'].apply(y_gtp)
test_df['gtp_cate'] = test_df['Gtp'].apply(y_gtp)

In [ ]:
value_counts = train_df['Gtp'].value_counts().sort_index()
plt.figure(figsize=(12, 6))
plt.bar(value_counts.index, value_counts.values)

In [ ]:
value_counts = train_df['AST'].value_counts().sort_index()
plt.figure(figsize=(12, 6))
plt.bar(value_counts.index, value_counts.values)

In [ ]:
def check_ast_alt(x):
    if x > 8 and x < 45:
        return 1 
    else:
        return 0
train_df['ast_cate'] = train_df["AST"].apply(check_ast_alt)
train_df['alt_cate'] = train_df["ALT"].apply(check_ast_alt)

test_df['ast_cate'] = test_df["AST"].apply(check_ast_alt)
test_df['alt_cate'] = test_df["ALT"].apply(check_ast_alt)

In [ ]:
cols = ['ast_cate','alt_cate','gtp_cate']
train_df = pd.get_dummies(data = train_df, columns=cols, drop_first=True)
test_df = pd.get_dummies(data = test_df, columns=cols, drop_first=True)

In [ ]:
train_df.drop(columns = ['id', 'smoking'], axis = 1).nunique()

In [ ]:
cols = ['dental caries','Urine protein','hearing(left)','hearing(right)']
train_df = pd.get_dummies(data = train_df, columns=cols, drop_first=True)
test_df = pd.get_dummies(data = test_df, columns=cols, drop_first=True)

In [ ]:
# Modeling

In [ ]:
from sklearn.model_selection import train_test_split
X = train_df.drop(['id','smoking'],axis=1)
y = train_df['smoking']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train_logt = np.log1p(X_train)
X_test_logt = np.log1p(X_test)

In [ ]:
params = {'learning_rate': 0.0192025093223225293453, 
            'colsample_bytree': 0.21329015151846925,
            'colsample_bylevel': 0.6748369225084079,
            'subsample': 0.8831564960046078,
            'reg_alpha': 1.1496763786731952e-05, 
            'reg_lambda': 7.512814356733987e-07, 
            'max_depth': 14, 
            'n_estimators': 1500,
            'min_child_weight': 21,
          'tree_method': 'gpu_hist',
          'eval_metric': 'auc',
          'booster': 'gbtree',
          'n_jobs': -1,
          'verbosity': 0}
params['tree_method'] = 'hist'  # Use CPU for training
xgb_model = xgb.XGBClassifier(**params)

xgb_model.fit(X_train_logt,y_train)
predictions_xgb = xgb_model.predict_proba(X_test_logt)[:, 1]
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test,predictions_xgb)
roc_auc
#0.870410157711284

In [ ]:
lgbm_classifier = LGBMClassifier(
    n_estimators=2000,
    max_depth=13,
    subsample=0.4796,
    num_leaves=120,
    learning_rate=0.023512,
    colsample_bytree=0.273534,
    reg_alpha=1.1496763786731952e-05,
    reg_lambda=7.512814356733987e-07,
)
lgbm_classifier.fit(X_train_logt,y_train)
predictions_lgbm = lgbm_classifier.predict_proba(X_test_logt)[:, 1]
roc_auc = roc_auc_score(y_test,predictions_lgbm)
roc_auc

In [ ]:
hist_gb_classifier = HistGradientBoostingClassifier(l2_regularization = 0.065,
                                             early_stopping = False,
                                             learning_rate = 0.07,
                                             max_iter = 300,
                                             max_depth = 11,
                                             max_bins = 255,
                                             min_samples_leaf = 25,
                                             max_leaf_nodes = 60)

hist_gb_classifier.fit(X_train_logt,y_train)

predictions_hist_gb = hist_gb_classifier.predict_proba(X_test_logt)[:, 1]
roc_auc = roc_auc_score(y_test, predictions_hist_gb)
roc_auc

In [ ]:
ensemble_predictions = 0.45 * predictions_lgbm + 0.85 * predictions_xgb + 0.15 * predictions_hist_gb

# Calculate the ROC AUC score for the ensemble
ensemble_roc_auc = roc_auc_score(y_test, ensemble_predictions)

print("Ensemble ROC AUC Score:", ensemble_roc_auc)

In [ ]:
# An accuracy of 0.87101 is obtained, You could further tune to get better resutls. I have assigned random weights, you can further do cross-val,etc.

In [ ]:
# The whole data to train:

test_df = test_df.drop('id',axis=1)

In [ ]:
print('The shape of training', train_df.shape) 
print('The shape of testing', test_df.shape)

In [ ]:
X_ = train_df.drop(['id','smoking'],axis=1) 
y_to_train = train_df['smoking']

#LogT:
X_to_train = np.log1p(X_)
X_to_test = np.log1p(test_df)

In [ ]:
print('The shape of training', X_to_train.shape) 
print('The shape of testing', X_to_test.shape)

In [ ]:
def all_model(X_train_all_scaled,y_all,X_test_all_scaled):
    
    lgbm_classifier.fit(X_train_all_scaled,y_all)
    predictions_lgbm = lgbm_classifier.predict_proba(X_test_all_scaled)[:, 1]
    
    xgb_model.fit(X_train_all_scaled,y_all)
    predictions_xgb = xgb_model.predict_proba(X_test_all_scaled)[:, 1]
    
    hist_gb_classifier.fit(X_train_all_scaled,y_all)
    predictions_hist = hist_gb_classifier.predict_proba(X_test_all_scaled)[:, 1]
    
    ensemble_predictions = 0.45 * predictions_lgbm + 0.85 * predictions_xgb + 0.15 * predictions_hist 
    return ensemble_predictions

In [ ]:
# CALLING THE ABOVE FUNCTION: 
predictions_ = all_model(X_to_train, y_to_train, X_to_test)

In [ ]:
len(predictions_)

In [ ]:
predictions = pd.DataFrame(predictions_)
submission = pd.read_csv('/Users/raekkkky/Desktop/DNA/Semester_2023_2/5주차/sample_submission.csv')
combined_df_ann = pd.concat([submission, predictions], axis=1)
combined_df_ann = combined_df_ann.drop('smoking',axis=1)
combined_df_ann.columns=['id','smoking']
combined_df_ann.to_csv('/Users/raekkkky/Desktop/DNA/Semester_2023_2/7주차/sample_submission(1).csv',index=False)

In [ ]:
combined_df_ann

# 코드공유 4

In [3]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import gc
import copy

from datetime import datetime, time, date, timedelta
from dateutil.relativedelta import * 

In [4]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pytorch_lightning as pl
rand_seed = 6
pl.seed_everything(seed = rand_seed) #utilities.seed 

Seed set to 6


6

In [6]:
train_file = r"/Users/raekkkky/Desktop/DNA/Semester_2023_2/5주차/train.csv"
test_file = r"/Users/raekkkky/Desktop/DNA/Semester_2023_2/5주차/test.csv"
submission_file = r"/Users/raekkkky/Desktop/DNA/Semester_2023_2/5주차/sample_submission.csv"

In [7]:
df_train = pd.read_csv(train_file)

In [8]:
df_train.shape

(159256, 24)

In [9]:
df_train.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,87,94,172,300,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,83,147,194,55,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,75,79,178,197,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,88,91,180,203,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,76,91,155,87,44,93,15.4,1,0.8,19,13,17,0,1


In [10]:
df_train.columns

Index(['id', 'age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)',
       'eyesight(right)', 'hearing(left)', 'hearing(right)', 'systolic',
       'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride',
       'HDL', 'LDL', 'hemoglobin', 'Urine protein', 'serum creatinine', 'AST',
       'ALT', 'Gtp', 'dental caries', 'smoking'],
      dtype='object')

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   159256 non-null  int64  
 1   age                  159256 non-null  int64  
 2   height(cm)           159256 non-null  int64  
 3   weight(kg)           159256 non-null  int64  
 4   waist(cm)            159256 non-null  float64
 5   eyesight(left)       159256 non-null  float64
 6   eyesight(right)      159256 non-null  float64
 7   hearing(left)        159256 non-null  int64  
 8   hearing(right)       159256 non-null  int64  
 9   systolic             159256 non-null  int64  
 10  relaxation           159256 non-null  int64  
 11  fasting blood sugar  159256 non-null  int64  
 12  Cholesterol          159256 non-null  int64  
 13  triglyceride         159256 non-null  int64  
 14  HDL                  159256 non-null  int64  
 15  LDL              

In [12]:
from sklearn.linear_model import LogisticRegression

import sklearn.metrics
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics 

In [13]:
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgb

In [14]:
x_cols = ['age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)',
       'eyesight(right)', 'hearing(left)', 'hearing(right)', 'systolic',
       'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride',
       'HDL', 'LDL', 'hemoglobin', 'Urine protein', 'serum creatinine', 'AST',
       'ALT', 'Gtp', 'dental caries', ]
y_col = 'smoking'

In [15]:
X = df_train[x_cols]
y = df_train[y_col]

In [16]:
skf = StratifiedKFold(n_splits=5, random_state = rand_seed, shuffle=True)
kfolds = []

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    kfolds.append([X.iloc[train_index,], X.iloc[test_index,], y.iloc[train_index,], y.iloc[test_index,]])

In [17]:
for i, (train_X, valid_X, train_y, valid_y) in enumerate(kfolds):
    print(i, train_X.shape, valid_X.shape, train_y.shape, valid_y.shape, train_y.mean(), valid_y.mean() )

0 (127404, 22) (31852, 22) (127404,) (31852,) 0.43736460393708204 0.4373665703880447
1 (127405, 22) (31851, 22) (127405,) (31851,) 0.43736117106863936 0.4373803020313334
2 (127405, 22) (31851, 22) (127405,) (31851,) 0.43736117106863936 0.4373803020313334
3 (127405, 22) (31851, 22) (127405,) (31851,) 0.437369020054158 0.4373489058428307
4 (127405, 22) (31851, 22) (127405,) (31851,) 0.437369020054158 0.4373489058428307


In [18]:
df_test = pd.read_csv(test_file)

In [19]:
dtrain = lgb.Dataset(X, label=y)

preds_list = []
model_list = []

params = {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'random_seed': 6, 'lambda_l1': 3.1422772805786794, 'lambda_l2': 0.3912821668022086, 'num_leaves': 93, 'feature_fraction': 0.24955364718656628, 'bagging_fraction': 0.9712078900905778, 'bagging_freq': 7, 'min_child_samples': 200, 'num_boost_round': 570, 'learning_rate': 0.049}

for seed_i in [1,2,3,5, 8, rand_seed,1024, 9999]:
    params_i = copy.deepcopy(params)
    params_i["random_seed"] = seed_i

    lgb_clf = lgb.train(params_i, dtrain)
    pred_prob = lgb_clf.predict(X)
    auc_val = metrics.roc_auc_score(y, pred_prob)
    print(seed_i, auc_val)
    model_list.append(copy.deepcopy(lgb_clf))
    
    pred_prob = lgb_clf.predict(df_test[x_cols])
    preds_list.append(pred_prob)

1 0.9082185154776216
2 0.9083433275754027
3 0.9083996338355262
5 0.9082099812691581
8 0.9082776048930501
6 0.9085796342847055
1024 0.9086962826204863
9999 0.9087952155398514


In [20]:
df_submission = pd.DataFrame(data={'id':df_test['id'], 'smoking':np.mean(preds_list, axis=0)})

df_submission.head()

,id,smoking
0,159256,0.680613
1,159257,0.218180
2,159258,0.389726
3,159259,0.017271
4,159260,0.611637


In [21]:
df_submission.to_csv('/Users/raekkkky/Desktop/DNA/Semester_2023_2/7주차/submission(2).csv',index = False)